In [1]:

import os

In [2]:
%pwd

'c:\\Users\\viswa\\Documents\\MLOPs\\MLOPs_Lung_Disease_Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\viswa\\Documents\\MLOPs\\MLOPs_Lung_Disease_Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int
    params_dropout: float


In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES,
            params_dropout=self.params.DROPOUT,

        )

        return prepare_base_model_config


In [8]:
from tensorflow.keras.applications import EfficientNetB0
import tensorflow as tf
from pathlib import Path

In [9]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        self.model = None
        self.full_model = None

    def get_base_model(self):
        """
        Load the EfficientNetB0 base model without top layers.
        """
        print("[INFO] Loading base EfficientNetB0 model...")
        self.model = EfficientNetB0(
            input_shape=tuple(self.config.params_image_size + [3]),
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )
        print("[INFO] Base model loaded successfully.")

    @staticmethod
    def _prepare_full_model(model, classes, dropout, freeze_all, freeze_till, learning_rate):
        """
        Add custom classification head and compile the model.
        """
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif freeze_till is not None and freeze_till > 0:
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        x = tf.keras.layers.GlobalAveragePooling2D()(model.output)
        x = tf.keras.layers.Dropout(dropout)(x)
        output = tf.keras.layers.Dense(units=classes, activation="softmax")(x)

        full_model = tf.keras.models.Model(inputs=model.input, outputs=output)

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.SparseCategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        print("[INFO] Full model compiled and ready for saving.")
        return full_model

    def update_base_model(self):
        """
        Adds classification head to base model and saves the full model.
        """
        if self.model is None:
            raise ValueError("Base model has not been loaded. Call get_base_model() first.")

        print("[INFO] Updating base model with classification head...")
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            dropout=self.config.params_dropout,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        print(f"[INFO] Saving full model to: {self.config.updated_base_model_path}")
        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        """
        Saves the model in TensorFlow SavedModel format (folder-based).
        Avoids JSON serialization issues with EagerTensors.
        """
        path = str(path)
        # Ensure path is folder (not .keras/.h5 file)
        if path.endswith(".keras") or path.endswith(".h5"):
            path = path.replace(".keras", "").replace(".h5", "")

        model.save(path)  # SavedModel format
        print(f"[INFO] Model saved successfully at {path} (SavedModel format)")


In [10]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
    print("[INFO] Prepare base model stage completed successfully.")
except Exception:
    print("[ERROR] Prepare base model stage failed.")


[2025-07-10 20:47:58,085: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-07-10 20:47:58,089: INFO: common: yaml file: params.yaml loaded successfully]
[2025-07-10 20:47:58,091: INFO: common: created directory at: artifacts]
[2025-07-10 20:47:58,093: INFO: common: created directory at: artifacts/prepare_base_model]
[INFO] Loading base EfficientNetB0 model...
[INFO] Base model loaded successfully.
[INFO] Updating base model with classification head...
[INFO] Full model compiled and ready for saving.
[INFO] Saving full model to: artifacts\prepare_base_model\base_model_updated
[2025-07-10 20:48:22,014: WARNING: save: Found untraced functions such as _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op, _jit_compiled_convolution_op while saving (showing 5 of 82). These functions will not be directly callable after loading.]
[2025-07-10 20:48:41,564: INFO: builder_impl: Assets written to: artifacts\prep